In [49]:
import torch
from onnx_tf.backend import prepare
import onnx
import tensorflow as tf

In [50]:
loaded = torch.jit.load('7x7/best.pt')
# loaded = torch.jit.load('5x5/best.pt')
# loaded = torch.jit.load('src/dualnet5x5_v2.pt')
# loaded = torch.jit.load('src/dualnet5x5_se.pt')
# loaded = torch.jit.load('src/dualnet7x7_se.pt')
print(loaded)

RecursiveScriptModule(
  original_name=DualNet
  (conv1): RecursiveScriptModule(original_name=Conv2d)
  (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
  (relu): RecursiveScriptModule(original_name=ReLU)
  (layers): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(
      original_name=BasicBlock
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
      (relu): RecursiveScriptModule(original_name=ReLU)
      (conv2): RecursiveScriptModule(original_name=Conv2d)
      (bn2): RecursiveScriptModule(original_name=BatchNorm2d)
      (downsample): RecursiveScriptModule(original_name=Sequential)
    )
    (1): RecursiveScriptModule(
      original_name=BasicBlock
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
      (relu): RecursiveScriptModule(original_name=ReLU)
      (conv2): RecursiveScriptModule(original_nam

In [51]:
loaded.eval()
bisze = 7
dummy_input = torch.randn((8, 12, bisze, bisze))
torch.onnx.export(loaded, dummy_input, "dualnet.onnx", example_outputs=torch.rand((1, bisze*bisze+2)), verbose=True)

graph(%x.1 : Float(8, 12, 7, 7, strides=[588, 49, 7, 1], requires_grad=0, device=cpu),
      %pi.4.bias : Float(50, strides=[1], requires_grad=0, device=cuda:0),
      %pi.4.weight : Float(50, 1568, strides=[1568, 1], requires_grad=0, device=cuda:0),
      %v.4.bias : Float(32, strides=[1], requires_grad=0, device=cuda:0),
      %v.4.weight : Float(32, 147, strides=[147, 1], requires_grad=0, device=cuda:0),
      %v.6.bias : Float(1, strides=[1], requires_grad=0, device=cuda:0),
      %v.6.weight : Float(1, 32, strides=[32, 1], requires_grad=0, device=cuda:0),
      %197 : Float(32, 12, 3, 3, strides=[108, 9, 3, 1], requires_grad=0, device=cuda:0),
      %198 : Float(32, strides=[1], requires_grad=0, device=cuda:0),
      %200 : Float(32, 32, 3, 3, strides=[288, 9, 3, 1], requires_grad=0, device=cuda:0),
      %201 : Float(32, strides=[1], requires_grad=0, device=cuda:0),
      %203 : Float(32, 32, 3, 3, strides=[288, 9, 3, 1], requires_grad=0, device=cuda:0),
      %204 : Float(32, st

In [52]:
tf.autograph.experimental.do_not_convert
onnx_model = onnx.load('dualnet.onnx')

tf_rep = prepare(onnx_model, device='gpu')
tf_rep.export_graph('model.pb')

INFO:tensorflow:Assets written to: model.pb\assets


In [53]:
!cd train
!saved_model_cli show --dir ./model.pb --all

�w�肳�ꂽ�p�X��������܂���B



MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['x.1'] tensor_info:
        dtype: DT_FLOAT
        shape: (8, 12, 7, 7)
        name: serving_default_x.1:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_0'] tensor_info:
        dtype: DT_FLOAT
        shape: (8, 51)
        name: PartitionedCall:0
  Method name is: tensorflow/serving/predict

Defined Functions:
  Function Name: '__call__'
        Named Argument #1
          x.1

  Function Name: 'gen_tensor_dict'


In [54]:

!tensorflowjs_converter --input_format=tf_saved_model --output_node_names='output_0' --saved_model_tags=serve model.pb js_model --quantize_uint8

Traceback (most recent call last):
  File "c:\users\sen\anaconda3\envs\py38\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\users\sen\anaconda3\envs\py38\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\sen\anaconda3\envs\py38\Scripts\tensorflowjs_converter.exe\__main__.py", line 4, in <module>
  File "c:\users\sen\anaconda3\envs\py38\lib\site-packages\tensorflowjs\__init__.py", line 21, in <module>
    from tensorflowjs import converters
  File "c:\users\sen\anaconda3\envs\py38\lib\site-packages\tensorflowjs\converters\__init__.py", line 21, in <module>
    from tensorflowjs.converters.converter import convert
  File "c:\users\sen\anaconda3\envs\py38\lib\site-packages\tensorflowjs\converters\converter.py", line 37, in <module>
    from tensorflowjs.converters import tf_saved_model_conversion_v2
  File "c:\users\sen\anaconda3\envs\py38\lib\site-packages\tensorflowjs\converters\tf_saved_model_conv

In [55]:
!pip list

Package                  Version
------------------------ -------------------
absl-py                  0.12.0
aiohttp                  3.7.4
aiohttp-cors             0.7.0
aioredis                 1.3.1
astor                    0.8.1
astunparse               1.6.3
async-timeout            3.0.1
attrs                    21.2.0
autopep8                 1.5.7
backcall                 0.2.0
bayesian-optimization    1.2.0
blessings                1.7
blinker                  1.4
brotlipy                 0.7.0
cachetools               4.2.2
certifi                  2020.12.5
cffi                     1.14.5
chardet                  3.0.4
click                    8.0.1
cma                      3.0.3
colorama                 0.4.4
colorful                 0.5.4
coverage                 5.5
cryptography             2.9.2
cycler                   0.10.0
Cython                   0.29.23
decorator                5.0.9
filelock                 3.0.12
future                   0.18.2
gast             